<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/hydrafloods/training_materials/oct_2021_hf_training/notebooks/scaling_workflows_day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scaling workflows using HYDRAFloods

In this notebook we will look at further use of HYDRAFloods for creating flood products over large areas or long time periods. We will discuss considerations and small nuasces for running processing over large number of images.

HYDRAFloods Python package can be found at: https://servir-mekong.github.io/hydra-floods/

## Setup
Before running the notebook, please mount your Google Drive to the notebook. We will use Google Drive to securely store Earth Engine credentials for use in other notebooks. This will allow us to bypass authenticating everytime saving time throughout the training.

In [ ]:
# mount the google drive so that we can save credentials
from google.colab import drive
drive.mount('/content/drive')

Now we will install the `hydrafloods` package for surface water mapping and `geemap` for interactive viewing results from Earth Engine.

You will get and error stating "*You must restart the runtime in order to use newly installed versions.*" This can be ignored.

In [ ]:
# install the packages needed
!pip install hydrafloods geemap

In [ ]:
import ee
import datetime
import hydrafloods as hf
import geemap.eefolium as geemap
import geemap.colormaps as cm

In [ ]:
_ = geemap.Map()

## Preventing errors with Optical processing

In [ ]:
region = ee.Geometry.Rectangle([-120,7,-77,32])
start_time = "2019-01-01"
end_time = "2019-07-01"

# get a Landsat 8 collection
lc8 = hf.Landsat8(region,start_time,end_time)

In [ ]:
lc8.n_images

In [ ]:
# lc8_errors = lc8.filter(ee.Filter.gt("CLOUD_COVER",90))

In [ ]:
mosaic = lc8.collection.median()

In [ ]:
optical_vis = {
    "min":50,
    "max":5500,
    "bands":"swir2,nir,green",
    "gamma":1.5,
}

In [ ]:
Map = geemap.Map(center=(15.5754, -89.8297), zoom=8)

Map.addLayer(mosaic,optical_vis, 'Landsat 8')

Map.addLayerControl()
Map

In [ ]:
# calculate water index
# here we calculate the modified normalized water index
water_index = lc8.apply_func(hf.mndwi)

In [ ]:
# extract water from each image
water = water_index.apply_func(hf.edge_otsu, initial_threshold=-0.9, edge_buffer=300,invert=True,thresh_no_data=0)


# thresh_no_data=0

In [ ]:
water_mosaic = water.collection.mode()

In [ ]:
Map = geemap.Map(center=(15.5754, -89.8297), zoom=8)

Map.addLayer(mosaic,optical_vis, 'Landsat 8')
Map.addLayer(water_mosaic.selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, 'Landsat 8 Water')

Map.addLayerControl()
Map

## Preventing errors with SAR processing

In [ ]:
region = hf.country_bbox("Dominican Republic")
# region = ee.Geometry.Point(-72.0154,16.8203,)

In [ ]:
s1 = hf.Sentinel1Asc(region,"2020-08-01","2020-10-01")

In [ ]:
s1.n_images

In [ ]:
merit = ee.Image("MERIT/Hydro/v1_0_1")

# extract out the DEM and HAND bands
dem = merit.select("elv").unmask(0)

# .unmask(0)

In [ ]:
# apply a (psuedo-) terrain flattening algorithm to S1 data
s1_flat = s1.apply_func(hf.slope_correction, elevation = dem, buffer = 100)

In [ ]:
# apply a speckle filter algorithm to S1 data
s1_filtered = s1_flat.apply_func(hf.gamma_map)

In [ ]:
# aggregate SAR observations to 30x30 m pixels
s1_aggregated = s1_filtered.apply_func(lambda x: x.focal_mean(40,"circle","meters").reproject(ee.Projection("EPSG:4326").atScale(30)))

In [ ]:
sar_vis = {
    "bands":"VV",
    "min":-25,
    "max":0
}

In [ ]:
Map = geemap.Map(center=(18.8959, -70.3015), zoom=8)

Map.addLayer(region,{},"Region of Interest")
Map.addLayer(s1.collection.mean(),sar_vis, 'Sentinel 1 mosaic')

Map.addLayerControl()
Map

In [ ]:
Map = geemap.Map(center=(18.8959, -70.3015), zoom=8)

Map.addLayer(region,{},"Region of Interest")
Map.addLayer(s1_aggregated.collection.mean(),sar_vis, 'Sentinel 1 mosaic')

Map.addLayerControl()
Map

## Ensuring processing runs

In [ ]:
region = ee.Geometry.Rectangle([-120,7,-77,32])
start_time = "2019-01-01"
end_time = "2020-06-01"

# get a Landsat 8 collection
s1 = hf.Sentinel1(region,start_time,end_time)

In [ ]:
# apply a (psuedo-) terrain flattening algorithm to S1 data
s1_flat = s1.apply_func(hf.slope_correction, elevation = dem, buffer = 100)

In [ ]:
# apply a speckle filter algorithm to S1 data
s1_filtered = s1_flat.apply_func(hf.gamma_map)

In [ ]:
# aggregate SAR observations to 30x30 m pixels
# only used for visualization
s1_aggregated = s1_filtered.apply_func(lambda x: x.focal_mean(40,"circle","meters").reproject(ee.Projection("EPSG:4326").atScale(30)))

In [ ]:
water = s1_filtered.apply_func(hf.edge_otsu, initial_threshold=-16, edge_buffer=300, scale=30)

In [ ]:
Map = geemap.Map(center=(15.5754, -89.8297), zoom=8)

Map.addLayer(s1.collection.mean(),sar_vis, 'Sentinel 1 mosaic')
Map.addLayer(water.collection.mode().selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, 'Sentinel 1 Water')

Map.addLayerControl()
Map

### Relaxing the computation scale

In [ ]:
water = s1_filtered.apply_func(hf.edge_otsu, initial_threshold=-16, edge_buffer=300, scale=300)

In [ ]:
Map = geemap.Map(center=(15.5754, -89.8297), zoom=8)

Map.addLayer(s1_aggregated.collection.mean(),sar_vis, 'Sentinel 1 mosaic')
Map.addLayer(water.collection.mode().selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, 'Sentinel 1 Water')

Map.addLayerControl()
Map

### Pipe functions

In [ ]:
region = ee.Geometry.Rectangle([-120,7,-77,32])
start_time = "2019-01-01"
end_time = "2021-01-01"

# get a Landsat 8 collection
s1 = hf.Sentinel1(region,start_time,end_time)

In [ ]:
process = (
    (hf.slope_correction, dict(elevation = dem.unmask(0), buffer = 100)),
    hf.gamma_map,
    (hf.edge_otsu, dict(initial_threshold=-16, edge_buffer=300, scale =120))
)

In [ ]:
water = s1.pipe(process)

In [ ]:
Map = geemap.Map(center=(15.5754, -89.8297), zoom=8)

Map.addLayer(s1_aggregated.collection.mean(),sar_vis, 'Sentinel 1 mosaic')
Map.addLayer(water.collection.mode().selfMask(),{"min":0,"max":1,"palette":cm.palettes.Blues}, 'Sentinel 1 Water')

Map.addLayerControl()
Map